In [1]:
import pandas as pd
import numpy as np

In [69]:
train=pd.read_csv("Titanic_train.csv")
test=pd.read_csv("Titanic_test.csv")

In [8]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [70]:
#Let's start by dropping the unnecessary columns 
train = train.drop(['PassengerId','Name','Ticket'], axis=1)
test = test.drop(['Name','Ticket'], axis=1)

In [10]:
#To know the number of missing values in our dataset
missing_values = train.isnull().sum()

In [11]:
missing_values

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [55]:
#So, there are 177 missing values in Age column and 687 missing values in Cabin Column and only 2 missing values in Embarked column
#We ll just try to fill those two embarked missing values of training dataset with most occured value i.e 'S'
train["Embarked"]= train["Embarked"].fillna("S")
train['Embarked'].head(20)

0     S
1     C
2     S
3     S
4     S
5     Q
6     S
7     S
8     S
9     C
10    S
11    S
12    S
13    S
14    S
15    S
16    Q
17    S
18    S
19    C
Name: Embarked, dtype: object

In [56]:
# now we will use "One hot Encoding" technique for Embarked Column
train_dummies  = pd.get_dummies(train['Embarked'])
train_dummies.drop(['S'], axis=1, inplace=True)
#print(train_dummies)

test_dummies  = pd.get_dummies(test['Embarked'])
test_dummies.drop(['S'], axis=1, inplace=True)

train = train.join(train_dummies)
test    = test.join(test_dummies)

In [58]:
# Now we will just drop the Embarked column 
train = train.drop(['Embarked'], axis=1)
test = test.drop(['Embarked'], axis=1)

In [21]:
# we will also drop the cabin column in our dataset ,which has a lot of missing values and wont be of much use for prediction
train = train.drop(['Cabin'], axis=1)
test = test.drop(['Cabin'], axis=1)

In [31]:
#Dealing with missing values in Age column
train['Age'] = train.groupby(['Pclass'])['Age'].transform(lambda x: x.fillna(x.mean()))
test['Age'] = test.groupby(['Pclass'])['Age'].transform(lambda x: x.fillna(x.mean()))

In [19]:
test_missing_values = test.isnull().sum()

In [20]:
test_missing_values

Pclass        0
Sex           0
Age          86
SibSp         0
Parch         0
Fare          0
Cabin       327
Embarked      0
dtype: int64

In [38]:
#There's only one missing value in Fare column in the test dataset, so we ll take care of it right now.
test["Fare"].fillna(test["Fare"].median(), inplace = True)

In [41]:
train.loc[ train['Fare'] <= 7.91, 'Fare'] = 0
train.loc[(train['Fare'] > 7.91) & (train['Fare'] <= 14.454), 'Fare'] = 1
train.loc[(train['Fare'] > 14.454) & (train['Fare'] <= 31), 'Fare'] = 2
train.loc[ train['Fare'] > 31, 'Fare'] = 3

In [40]:
test.loc[ test['Fare'] <= 7.91, 'Fare'] = 0
test.loc[(test['Fare'] > 7.91) & (test['Fare'] <= 14.454), 'Fare'] = 1
test.loc[(test['Fare'] > 14.454) & (test['Fare'] <= 31), 'Fare'] = 2
test.loc[test['Fare'] > 31, 'Fare'] = 3

In [42]:
train['Fare'] = train['Fare'].astype(int)
test['Fare'] = test['Fare'].astype(int)

In [27]:
#we will try to convert the fare and age columns into categorical 
#type(train['Age'])
type(test['Age'])

pandas.core.series.Series

In [32]:
# so , we will cnonvert Age to int

train['Age']= train['Age'].astype(int)

In [33]:
test['Age'] = test['Age'].astype(int)

In [34]:
train.loc[ train['Age'] <= 16, 'Age'] = 0
train.loc[(train['Age'] > 16) & (train['Age'] <= 32), 'Age'] = 1
train.loc[(train['Age'] > 32) & (train['Age'] <= 48), 'Age'] = 2
train.loc[(train['Age'] > 48) & (train['Age'] <= 64), 'Age'] = 3
train.loc[(train['Age'] > 64), 'Age'] = 4

In [35]:
test.loc[ test['Age'] <= 16, 'Age'] = 0
test.loc[(test['Age'] > 16) & (test['Age'] <= 32), 'Age'] = 1
test.loc[(test['Age'] > 32) & (test['Age'] <= 48), 'Age'] = 2
test.loc[(test['Age'] > 48) & (test['Age'] <= 64), 'Age'] = 3
test.loc[(test['Age'] > 64), 'Age'] = 4

In [44]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,1,1,0,0,S
1,1,1,female,2,1,0,0,C
2,1,3,female,1,0,0,0,S
3,1,1,female,2,1,0,0,S
4,0,3,male,2,0,0,0,S


In [45]:
#we will now try to create a new column called "Family" using "Parch" and "SibSp"
train['Family'] =  train["Parch"] + train["SibSp"]
train['Family'].loc[train['Family'] > 0] = 1
train['Family'].loc[train['Family'] == 0] = 0

/Users/saikrishna/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [46]:
test['Family'] =  test["Parch"] + test["SibSp"]
test['Family'].loc[test['Family'] > 0] = 1
test['Family'].loc[test['Family'] == 0] = 0

/Users/saikrishna/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [48]:
#Now , we will just drop off the "Parch" and "SibSp" columns
train = train.drop(['SibSp','Parch'], axis=1)
test = test.drop(['SibSp','Parch'], axis=1)

In [51]:
# we will also conver the "Sex" column into categorical
sexes = sorted(train['Sex'].unique())
genders_mapping = dict(zip(sexes, range(0, len(sexes) + 1)))
train['Sex'] = train['Sex'].map(genders_mapping).astype(int)
test['Sex'] = test['Sex'].map(genders_mapping).astype(int)

In [59]:
train.head()

,Survived,Pclass,Sex,Age,Fare,Family,age_class,C,Q
0,0,3,1,1,0,1,3,0,0
1,1,1,0,2,0,1,2,1,0
2,1,3,0,1,0,0,3,0,0
3,1,1,0,2,0,1,2,0,0
4,0,3,1,2,0,0,6,0,0


In [53]:
#we will create another column called age_class , using two other columns "Age" and "Pclass"
train['age_class'] = train['Age'] * train['Pclass']
test['age_class'] = test['Age'] * test['Pclass']

In [63]:
# Seems like our dataset is good to go :P
X_train= train.drop("Survived", axis=1)
Y_train = train['Survived']
X_test = test

In [62]:
test.head()

,Pclass,Sex,Age,Fare,Family,age_class,C,Q
0,3,1,2,0,0,6,0,1
1,3,0,2,0,1,6,0,0
2,2,1,3,0,0,6,0,1
3,3,1,1,0,0,3,0,0
4,3,0,1,0,1,3,0,0


In [61]:
%%HTML
<h3> RANDOM FOREST </h3>

In [64]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=100)
#random_forest = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=10, max_features='sqrt', min_samples_split=5)

random_forest.fit(X_train, Y_train)

Y_pred_rf = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)

0.8305274971941639

In [65]:
# Sweeeet, 83% , now let's try Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
gradient_boost = GradientBoostingClassifier(n_estimators=100)
gradient_boost.fit(X_train, Y_train)

Y_pred_2 = gradient_boost.predict(X_test)

gradient_boost.score(X_train, Y_train)

0.8294051627384961

In [67]:
# ummmmm.....i was hoping for 90% , so, 83% it is......  
Y_pred_rf

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,

In [71]:
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": Y_pred_rf
    })
submission.to_csv('titanic.csv', index=False)

In [76]:
from sklearn.naive_bayes import GaussianNB

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
gaussian.score(X_train, Y_train)

0.77665544332211

In [79]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)

knn.fit(X_train, Y_train)

Y_pred_KNN = knn.predict(X_test)

knn.score(X_train, Y_train)

0.8271604938271605

In [80]:
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": Y_pred_KNN
    })
submission.to_csv('titanicKNN.csv', index=False)